In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
!pip install jours-feries-france
from jours_feries_france import JoursFeries
from scipy.integrate import quad,romberg

In [228]:
# accès aux données : https://odre.opendatasoft.com/explore/dataset/eco2mix-national-cons-def/information

# dates de début et de fin de la plage de données, format AAAA-MM-JJ
date_debut = '2012-01-01'
date_fin = '2022-12-31'

# variables :
# perimetre ; nature ; date ; heure ; date_heure ;
# consommation ; prevision_j1 ; prevision_j ; fioul ; charbon ;
# gaz ; nucleaire ; eolien ; solaire ; hydraulique ;
# pompage ; bioenergies ; ech_physiques ; taux_co2 ; ech_comm_angleterre ;
# ech_comm_espagne ; ech_comm_italie ; ech_comm_suisse ; ech_comm_allemagne_belgique ; fioul_tac ;
# fioul_cogen ; fioul_autres ; gaz_tac ; gaz_cogen ; gaz_ccg ;
# gaz_autres ; hydraulique_fil_eau_eclusee ; hydraulique_lacs ; hydraulique_step_turbinage ; bioenergies_dechets ;
# bioenergies_biomasse ; bioenergies_biogaz

df_cons = pd.read_csv(r"data/eco2mix-national-cons-def.csv", sep=';')
# toutes les consommations sont en MW
#df_cons = df_cons[["Date", "Heure","Consommation"]].rename(mapper = {"Date" : "date","Heure" : "heure","Consommation" : "consommation"}, axis = 1)

/tmp/ipykernel_1314/777147624.py:17: DtypeWarning: Columns (23,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cons = pd.read_csv(r"data/eco2mix-national-cons-def.csv", sep=';')


In [229]:
mask = (df_cons['date'] >= date_debut) & (df_cons['date'] <= date_fin)
df_cons = df_cons.loc[mask]
df_cons.sort_values(["date","heure"], inplace=True)
# ajout d'une colonne avec les dates et les heures
df_cons['date_heure'] = pd.to_datetime(df_cons['date']+'T'+df_cons['heure'])
df_cons.dropna(axis = 0, inplace=True)
df_cons.reset_index(drop=True, inplace=True)

In [230]:
# calcul de la consommation moyenne par jour
cons_moy_quot = df_cons[['consommation']] \
    .groupby(df_cons['date_heure'].dt.normalize().rename('date')) \
    .mean()
#cons_moy_quot.plot()

In [231]:
cons_moy_quot_ann = cons_moy_quot \
    .assign(annee=cons_moy_quot.index.year, jour=cons_moy_quot.index.dayofyear) \
    .pivot(index='jour', columns='annee', values='consommation')
#cons_moy_quot_ann.plot()

In [232]:
#cons_moy_quot_ann.std(axis = 1)
#cons_moy_quot_ann.mean(axis = 1)
gauss_param = pd.concat([cons_moy_quot_ann.mean(axis = 1),cons_moy_quot_ann.std(axis = 1)**2],axis = 1)
gauss_param = gauss_param.rename(columns = {0 : 'mean',1 : 'var'})

In [233]:
# dataframe avec les consommations moyennes quotidiennes en MW
df_cons_moy_quot = pd.DataFrame(columns = ["date","consommation_moy (MW)"])
df_cons_moy_quot["date"] = cons_moy_quot.index
df_cons_moy_quot["consommation_moy (MW)"] = cons_moy_quot.values

In [234]:
# dataframe avec les consommations moyennes quotidiennes de 2020-2021
mask_16_17 = (df_cons_moy_quot['date'] >= "2016-09-01") & (df_cons_moy_quot['date'] <= "2017-08-31")
df_cons_16_17 = df_cons_moy_quot.loc[mask_16_17].reset_index(drop=True)

def date_to_wd(tmstp) : #Fonction de transformation d'un timestamp en jour de la semaine
    dt = tmstp.date()
    return(dt.weekday())

def feries(tmstp) : #Fonction d'évaluation de si un jour est férié ou non
    return(JoursFeries.is_bank_holiday(tmstp.date(), zone="Métropole"))

df_cons_16_17['weekday'] = df_cons_16_17['date'].apply(date_to_wd)
df_cons_16_17['ferie'] = df_cons_16_17['date'].apply(feries)

In [9]:
# Nombre de jours de chaque type
n_r = 22
n_w = 43
n_b = 300

# On converti les prix en €/MW
p_r = 0.6712*24*1000
p_w = 0.1508*24*1000
p_b = 0.1249*24*1000

In [101]:
#Initialisation de la récursivité

memory = {}

for i in range(n_b+1) :
    for j in range(n_w+1) :
        for k in range(n_r+1) :
            memory[(-1, j, k)] = -np.inf
            memory[(i, -1, k)] = -np.inf
            memory[(i, j, -1)] = -np.inf
        
memory[(0,0,0)] = 0

def g(i,j,k) :
    """
        Entrées : i,j,k respectivement les nombre de jours bleus, blancs et rouges restants à attribuer
        Sortie : le gain espéré sur les prochains jours
    """
    if i+j+k < 0 or i < 0 or j < 0 or k < 0 :
        memory[(i,j,k)] = -np.inf
        return -np.inf
    try :
        return memory[(i,j,k)]
    except :
        day = 366-i-j-k #Jours auquel on se trouve (les indexs commencent à 1)
        moy,var = gauss_param.loc[day]
        f = lambda x : max(max(p_b*x + g(i-1,j,k),p_w*x + g(i,j-1,k)),p_r*x + g(i,j,k-1)) * (1/np.sqrt(2*np.pi*var)) * np.exp(-(x-moy)**2/(2*var))
        esp = quad(f,moy-5*np.sqrt(var),moy+5*np.sqrt(var))[0]
        memory[(i,j,k)] = esp
        return esp    

In [122]:
# Backward induction

#Initialisation des variables de jours restants
rem_b = n_b
rem_w = n_w
rem_r = n_r

#Liste des types de jours
type_jours = []

for k in range(len(df_cons_16_17)) :
    #Premier cas : c'est dimanche (forcément un jour bleu)
    if (df_cons_16_17.loc[i,'weekday'] == 6) :
        types_jours.append['bleu']
        rem_b -= 1
        continue
        
    #Calcul des gains potentiels
    w_k = df_cons_16_17.loc[k,"consommation_moy (MW)"]
    
    gb = p_b*w_k + g(rem_b-1,rem_w,rem_r)
    gw = p_w*w_k + g(rem_b,rem_w-1,rem_r)
    gr = p_r*w_k + g(rem_b,rem_w,rem_r-1)
    
    #Deuxième cas : c'est un jour férié ou nous sommes en dehors de entre le 1er novembre et le 31 mars (arbitrage jour bleu jour blanc)
    if df_cons_16_17.loc[i,'ferie'] or (df_cons_16_17.loc[i,'date'].to_pydatetime() < datetime.datetime(2016,11,1,0,0) and df_cons_16_17.loc[i,'date'].to_pydatetime() > datetime.datetime(2017,3,31,0,0)) :
        if gb >= gw : #on préfère utiliser un jour bleu qu'un jour blanc
            rem_b -= 1
            type_jours.append('bleu')
            continue
        else :
            rem_w -= 1
            type_jours.append('blanc')
            continue

    #Troisième cas : arbitrage jours bleus, blancs et rouges
    if gb >= gw and gb >= gr : #on préfère utiliser les jours bleus en premier
        rem_b -= 1
        type_jours.append('bleu')
    elif gw >= gb and gw >= gr : #puis les blancs
        rem_w -= 1
        type_jours.append('blanc')
    elif gr >= gb and gr >= gw : #et enfin les rouges
        rem_r -= 1
        type_jours.append('rouge')
    

In [235]:
#Calcul du gain par la backward induction

df_cons_16_17['backward'] = type_jours
df_cons_16_17['gain backward'] = df_cons_16_17.replace({'backward':{'bleu' : p_b,'blanc':p_w,'rouge':p_r}})['backward']*df_cons_16_17['consommation_moy (MW)']

In [236]:
ind_proph_r = df_cons_16_17[(~df_cons_16_17['weekday'].isin([5,6]))&(~df_cons_16_17.ferie)].nlargest(n_r,'consommation_moy (MW)').index
ind_proph_w = df_cons_16_17[(~df_cons_16_17['weekday'].isin([5]))&(~df_cons_16_17.index.isin(ind_proph_r))].nlargest(n_w,'consommation_moy (MW)').index
ind_proph_b = df_cons_16_17[~df_cons_16_17.index.isin(list(ind_proph_r)+list(ind_proph_w))].index

In [250]:
prophet = pd.Series(data = n_r*['rouge']+n_w*['blanc']+n_b*['bleu'],index = list(ind_proph_r)+list(ind_proph_w)+list(ind_proph_b))
df_cons_16_17['prophet'] = prophet
df_cons_16_17['gain prophet'] = df_cons_16_17.replace({'prophet':{'bleu' : p_b,'blanc':p_w,'rouge':p_r}})['prophet']*df_cons_16_17['consommation_moy (MW)']

In [254]:
gain_back = df_cons_16_17['gain backward'].sum()
gain_proph = df_cons_16_17['gain prophet'].sum()
print('le gain de la backward induction est : ',gain_back,'\n le gain du prophet est : ',gain_proph,'\n le competitive ratio est : ',gain_back/gain_proph) 

le gain de la backward induction est :  82630397692.30002 
 le gain du prophet est :  84529622581.75 
 le competitive ratio est :  0.9775318423122827


In [259]:
#comparaison avec EDF
tempo_16_17 = pd.read_csv(r"data/eCO2mix_RTE_tempo_2016-2017.csv", sep=";", encoding='latin-1')
tempo_16_17.rename({"Date": "date"},axis=1,inplace=True)
check = pd.concat([tempo_16_17, df_cons_16_17],axis=1)
rouge_16_17 = check[check["Type de jour TEMPO"] == "ROUGE"]
blanc_16_17 = check[check["Type de jour TEMPO"] == "BLANC"]
bleu_16_17 = check[check["Type de jour TEMPO"] == "BLEU"]

gain_rouge_reel = rouge_16_17["consommation_moy (MW)"].sum() * p_r
gain_blanc_reel = blanc_16_17["consommation_moy (MW)"].sum() * p_b
gain_bleu_reel = bleu_16_17["consommation_moy (MW)"].sum() * p_b
gain_reel = gain_rouge_reel+gain_blanc_reel+gain_bleu_reel

print('EDF a gagné sur la même période : ',gain_reel,'\n la backward induction a gagné : ',gain_back,'\n soit, ',(gain_back-gain_reel)/gain_reel,'% de plus')

EDF a gagné sur la même période :  81211878825.9 
 la backward induction a gagné :  82630397692.30002 
 soit,  0.017466888919550918 % de plus
